In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import datetime as dt
import os.path
import backtrader as bt
import numpy as np
from pprint import pformat
import pandas as pd
import matplotlib.pyplot as plt
import datetime

In [ ]:
import sys
sys.path.append('..')
from decisivealpha.montecarlo import MonteCarlo

## Strategy

In [ ]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('emaSperiod', 7),
        ('emaLperiod', 15),
        ('printlog', False),
        ('printstop', False)
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        self.ema_short = bt.indicators.ExponentialMovingAverage(
                            self.datas[0], period=self.params.emaSperiod)
        self.ema_long = bt.indicators.ExponentialMovingAverage(
                            self.datas[0], period=self.params.emaLperiod) 

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.ema_short[0] > self.ema_long[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.ema_short[0] < self.ema_long[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        pass
        #self.log('emaperiod:%2d %2d, %.2f' %
        #         (self.params.emaSperiod, self.params.emaLperiod, self.broker.getvalue()), doprint=True)

## Analyzers

In [ ]:
class DecisiveAnalyzer(bt.Analyzer):
    
    def __init__(self):
        self.emaSperiod = self.strategy.params.emaSperiod
        self.emaLperiod = self.strategy.params.emaLperiod
        self.equity = []
        self.equitydf = pd.DataFrame()

    def start(self):
        # Not needed ... but could be used
        self.init_cash = self.strategy.broker.cash 
        self.num_trades = 0
        self.first_trade_open = None

    def next(self):
        # Not needed ... but could be used
        pass

    def notify_trade(self, trade):
        if not self.first_trade_open:
            self.first_trade_open = self.strategy.datetime.datetime()
            self.equitydf.at[self.first_trade_open, 'equity'] = self.init_cash

        if trade.isclosed:
            self.num_trades += 1
            self.equitydf.at[self.strategy.datetime.datetime(), 'equity'] = self.strategy.broker.getvalue()
            
    def stop(self):
        self.final_cash = self.strategy.broker.cash
        self.final_val = self.strategy.broker.get_value()

    def get_analysis(self):
        # Add trade profit to dataframe
        self.equitydf['net_profit'] = self.equitydf['equity'].diff()

        outp = {
            'params': (self.emaSperiod, self.emaLperiod),
            'profit': self.final_val - self.init_cash,
            'num_trades': self.num_trades,
            'equitydf': self.equitydf,
        }
        
        return outp
    

In [ ]:
def best_result_from_cerebro_opti_run(result):
    params  = []
    n_trades = []
    profit   = []
    trades   = []
    for res in result:
        r = res[0].analyzers.decisive.get_analysis()
        params.append(r['params'])
        n_trades.append(r['ntrade'])
        profit.append(r['profit'])
        trades.append(r['trades'])

    prof_ind = np.argmax(profit) 
    best_params = params[prof_ind]
    best_profit = profit[prof_ind]
    best_ntrades = n_trades[prof_ind]
    best_trades = trades[prof_ind]
    
    print('best:{} profit:{} trades:{}'.format(best_params, best_profit, best_ntrades))
    return (best_params, best_profit, best_ntrades, best_trades)

In [ ]:
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()
df[:]['close'].plot()

## In-sample Parameters

In [ ]:
def get_wf_dates(data, nday_train, nday_test):
    periods = []
    start = data.index[0]
    end = data.index[-1]

    n = 0
    is_start  = start
    while (end - is_start).days >= (nday_train + nday_test):

        is_end = is_start + dt.timedelta(days=nday_train)
        oos_start = is_end
        oos_end = oos_start + dt.timedelta(days=nday_test)

        periods.append({
                        'run': n, 
                        'oos': (oos_start, oos_end), 
                        'is': (is_start, is_end)
                        })
        n += 1
        is_start += dt.timedelta(days=nday_test)
    
    return periods

In [ ]:
nday_train = 365
nday_test  = 90
periods = get_wf_dates(df, nday_train, nday_test)

start_date = periods[0]['is'][0]
end_date = periods[-1]['is'][1]
print('Start/End: {} - {}'.format(start_date, end_date))

cerebro = bt.Cerebro()

strats = cerebro.optstrategy(
            TestStrategy,
            emaSperiod=range(4, 8, 1),
            emaLperiod=range(25, 35, 1))

cerebro.addanalyzer(DecisiveAnalyzer, _name='decisive')

cerebro.optreturn = False
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
cerebro.broker.setcommission(commission=0.0)

data = bt.feeds.PandasData(dataname = df[start_date:end_date])
cerebro.adddata(data)

results = cerebro.run(maxcpus=1)   

## Out-of-sample Parameters

In [ ]:
# Get in-sample and out-of-sample best parameters with correct pre-allocation
for period in periods:
    # Get the insample time period
    insample_start = period['is'][0]
    insample_end = period['is'][1]
    print('In-Sample Period: {} - {}'.format(insample_start, insample_end))
    
    # Put the best parameters and best profit into a list
    period_netprofit = []
    for result in results:
        r = result[0].analyzers.decisive.get_analysis()
        df = r['equitydf']
        maperiod = r['params']
        period_profit = df[insample_start:insample_end]['net_profit'].sum()
        #print('\tmaperiod {}: {}'.format(maperiod, period_profit))
        period_netprofit.append((maperiod, period_profit))
        
    # Get the best profit parameters
    max_set = max(period_netprofit, key=lambda x:x[1])
    max_maperiod, max_profit = max_set
    
    # Add it to the dictionary
    print('\t\tBest is: {} with {} profit'.format(max_maperiod, max_profit))
    period['best_insample_param'] = max_maperiod
    period['best_insample_profit'] = max_profit

## Walkforward Efficiency

In [ ]:
def days_from_date_tuple(mytuple):
    """Get days between dates to annualize"""
    days_start = mytuple[0]
    days_end = mytuple[1]
    days = (days_end - days_start).days
    return days

# Get in-sample and out-of-sample best parameters with correct pre-allocation
all_walkforward_efficiency = []
for period in periods:
    # Get the insample time period
    oos_start = period['oos'][0]
    oos_end = period['oos'][1]
    insample_profit = period['best_insample_profit']
    print('Out-of-Sample Period: {} - {}'.format(oos_start, oos_end))
    
    # Find the set of results that has the same best parameters
    target_result = None
    for result in results:
        r = result[0].analyzers.decisive.get_analysis()
        if r['params'] == period['best_insample_param']:
            target_result = r
            break
            
    # Get the equity curve of this best parameter
    target_df = target_result['equitydf']
    
    # Splice to the OOS date range only
    oos_profit = target_df[oos_start:oos_end]['net_profit'].sum()
    oos_trades = target_df[oos_start:oos_end].shape[0]
    
    # Annualize the profit
    insample_days = days_from_date_tuple(period['is'])
    oos_days = days_from_date_tuple(period['oos'])

    insample_annual_profit = 365/insample_days * insample_profit
    oos_annual_profit = 365/oos_days * oos_profit
 
    # Calculate WFE
    if oos_annual_profit != 0:
        period_wfe = insample_annual_profit * 100 / oos_annual_profit
    else:
        period_wfe = 0
    period['wfe'] = period_wfe
    all_walkforward_efficiency.append(period_wfe)
    
    period['oos_trades'] = oos_trades
    
    print('\t Period Profit IS:{:0.2f} / OOS:{:0.2f}'.format(insample_profit, oos_profit))
    print('\t Annual Profit IS:{:0.2f} / OOS:{:0.2f}'.format(insample_annual_profit, oos_annual_profit))

    print('\t WFE: {:0.1f}%'.format(period_wfe))
    print('\t oos trades: {}'.format(oos_trades))

wfe = sum(all_walkforward_efficiency) / len(all_walkforward_efficiency) 
print('Overall WFE: {:0.1f}%'.format(wfe))

## Walkforward Equity Curve

In [ ]:
# Create a Stratey
class WalkforwardStrategy(bt.Strategy):

    params = (
        ('emaSperiod', 7),
        ('emaLperiod', 15),
        ('printlog', False),
        ('printstop', False),
        ('walkforward', True),
        ('live', False)
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        self.ema_short = bt.indicators.ExponentialMovingAverage(
                            self.datas[0], period=self.params.emaSperiod)
        self.ema_long = bt.indicators.ExponentialMovingAverage(
                            self.datas[0], period=self.params.emaLperiod) 
        
        # Trim the indicators if we are running live, just need the current one
        if not self.params.live:
            pass
        
        
        self.current_row = None
        self.wfsma = []
        
        if self.params.walkforward:
            for run in self.params.walkforward:
                self.wfsma.append({
                    'emaS': bt.indicators.ExponentialMovingAverage(self.datas[0], period=run['best_insample_param'][0]),
                    'emaL': bt.indicators.ExponentialMovingAverage(self.datas[0], period=run['best_insample_param'][1]),
                    'test_period': run['oos'],
                    'train_param': run['best_insample_param'],
                }) 

        print(pformat(self.wfsma))
        
        

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):

        # Walk-forward logic
        for row in self.wfsma:
            start, end = row['test_period']
            period_start = start
            period_end = end
            if self.datetime.datetime() >= period_start and self.datetime.datetime() < period_end:
                self.current_row = row
        
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.current_row['emaS'][0] > self.current_row['emaL'][0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.current_row['emaS'][0] < self.current_row['emaL'][0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('emaperiod:%2d %2d, %.2f' %
                 (self.params.emaSperiod,self.params.emaLperiod, self.broker.getvalue()), doprint=True)

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(WalkforwardStrategy, 
                    emaSperiod=15, 
                    emaLperiod=15, 
                    walkforward=periods,
                    live=False)

# Load data
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()

# periods
start_date = periods[0]['oos'][0]
end_date = periods[-1]['oos'][0]

print('Start: {} End: {}'.format(start_date, end_date))
data = bt.feeds.PandasData(dataname = df[start_date:end_date])


# Add the Data Feed to Cerebro
cerebro.adddata(data)
cerebro.addanalyzer(DecisiveAnalyzer, _name='decisive')

# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
results = cerebro.run(maxcpus=1)

In [ ]:
# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
r = results[0].analyzers.decisive.get_analysis()
cumsum = r['equitydf']['net_profit'].cumsum()

plt.figure(figsize=(10,5))
plt.plot(cumsum)
plt.title('Walkforward Equity Curve')
plt.xlabel('Trades')
plt.ylabel('Equity')

In [ ]:
cerebro.plot(volume=False, iplot=False)

## Monte Carlo

In [ ]:
# Initialize
margin = 5000
mc = MonteCarlo(r['equitydf']['net_profit'].values[1:])

# We will sample with replacement the number of trades per year
# so we need the start and end date to determine how many trades at in a year on average
mc.settings(margin, start_date, end_date)

# Test different levels of equity starting at this value
trial_starting_equity = int(margin * 1.5)

# Run the Monte Carlo
results = mc.run(trial_starting_equity)

In [ ]:
# Put the results in a dataframe so it's nicer to look at in notebook
# Our goal is to get the highest equity below 10% Risk of Ruin
df = pd.DataFrame(index=range(1,len(results)))
count = 1
for result in results:
    df.loc[count, 'equity'] = result['equity']
    df.loc[count, 'is_ruined'] = result['is_ruined']
    df.loc[count, 'is_profitable'] = result['is_profitable']
    df.loc[count, 'returns_pct'] = result['returns_pct']
    df.loc[count, 'drawdown_pct'] = result['drawdown_pct']
    df.loc[count, 'returns_per_drawdown'] = result['returns_per_drawdown']
    count += 1

# Get the recommended values
recommended = df[df['is_ruined'] <= 10].iloc[0]
print('Recommend a starting equity of {}, which has {:0.2}% Risk-of-Ruin, \n\t{:0.0f}% Probability-of-Profit and a {:0.2f} Returns/Drawdown Ratio'.format(
                recommended['equity'], recommended['is_ruined'], 
                recommended['is_profitable'], recommended['returns_per_drawdown']))

if recommended['is_ruined'] > 10 or recommended['returns_per_drawdown'] < 2.0:
    print("Risk Assessment: FAILED")
else:
    print("Risk Assessment: PASSED")

mc_1p5x = recommended['drawdown_pct'] * 1.5
print("MC-Drawdown: {:0.1f}% MC-1.5x-DD: {:0.1f}%".format(recommended['drawdown_pct'], mc_1p5x))

profit = recommended['equity'] * recommended['returns_pct'] / 100
months = (end_date - start_date).days/30
average_monthly_net_profit = profit / months
print("Average monthly net profit: {:0.1f}".format(average_monthly_net_profit))

df